# Theory of BattleSimulator

As of **version 0.3.6** current patch.

This notebook covers the current theoretical underpinning of the BattleSimulator.

Let $\mathbf{u}_i = [x, y]$ define the 2-D cartesian positional coordinates of unit $i$.

## Assignment of Unit Positions

### Drawn from a normal distribution

A gaussian or normal distribution is a common continuous probability that can be used to represent real-valued random variables. Here we draw the 2-D positional vector $\mathbf{u} = [x, y]$ as:

\begin{align}
\mathbf{u} \sim \mathcal{N}(\mu, \Sigma) \tag{1}
\end{align}

where $\mu$ is the mean, and $\Sigma$ the covariance matrix.

### Drawn from a uniform distribution

A uniform distribution is a statistical distribution that bears an equal likelihood of any value in the range $[a, b]$ of being drawn:

\begin{align}
\mathbf{u} \sim \mathcal{U}(a, b) \tag{2}
\end{align}

## Computing the distance from a target unit/location

In order to move the units in 2D space, we have to define how far units are from each other when it comes to calculating whether they are in range to strike. This is determined by the positions and the `range` parameter given to each unit type.

The euclidean distance is defined mathematically as:

\begin{align*}
\mathcal{D}(\textbf{p}, \textbf{q}) = \mathcal{D}(\textbf{q}, \textbf{p}) = \sqrt{(q_1 - p_1)^2 + (q_2 - p_2)^2}
\tag{3}
\end{align*}

### Distance from closest enemy

For the primary targeting algorithm, this distance quantity is something to be minimized with respect to all other potential enemies:

\begin{align}
\arg \min \mathcal{D}_i(\mathbf{u}_i, \mathbf{u}_j) \quad \forall j \tag{4}
\end{align}


### Defining directional derivatives

The directional derivative $\nabla_{\textbf{u}} f(x_0, y_0)$ is the rate at which the function $f(x, y)$ changes at a point $(x_0, y_0)$ in the direction $\textbf{u}$. It is defined as:

\begin{align}
\nabla_{\mathbf{u}} f \equiv \nabla f \cdot \frac{\mathbf{u}}{|\mathbf{u}|} \\
= \lim_{h \to 0} \frac{f(\mathbf{x} + h \, \hat{\mathbf{u}}) - f(\mathbf{x})}{h}
\tag{5}
\end{align}

where $\mathbf{u}=[\mathbf{x}, \mathbf{y}]$. In our case, within *Euclidean space*, the directional derivative is respective to the vector $\mathbf{v}$ after normalization, so it's independent of it's magnitude and depends only on the direction.

### The normalized derivative

Since we exclude the magnitude from our calculation; we normalize for it:

\begin{align}
\delta \mathbf{u}_{i}=-\frac{\mathbf{u}_i - \mathbf{u}_j}{\mathcal{D}(\mathbf{u}_i, \mathbf{u}_j)} \tag{6}
\end{align}

This is essentially the *negative normalized direction* of the unit. We make it negative such that the unit moves *towards* the enemy rather than away. Note that it is **essential** that $\mathbf{u}_j - \mathbf{u}_i$ rather than $\mathbf{u}_i - \mathbf{u}_j$, otherwise the unit will move in the *other direction away from the target*.


## Unit movement towards an enemy

Let $s_i$ be the base speed multiplier of unit $i$, and let $z_i(t)$ be the z-coordinate of the height of unit $i$ at time $t$.

The change in unit position given time $t$, measured as $\dot{\mathbf{u}}$ is determined as:

\begin{align}
\dot{\mathbf{u}}_i(t) = \frac{d\mathbf{u}_i}{dt}(\mathbf{u}_i, \mathbf{u}_j) \approx s_i \, \delta \mathbf{u}_{i}\, \left(1-\frac{z_i(t)}{2} \right) \delta t \quad \forall i \\
\tag{7}
\end{align}

where $\delta t$ is the change in time step (default to 1), unit $j$ is the target that unit $i$ wishes to draw towards, and where $\delta \mathbf{u}_{i}$ is our normalized direction for unit $i$. In essense, this is simply the 'speed' of the unit multiplied by the negative direction of the opponent to move in that direction. Note that $ \mathbf{u}_j$ doesn't have to be another Unit, but could be a waypoint for moving a Unit to a particular landmark or location.

Further to this, larger values of $z_i(t)$ will slow down unit movement derivatives, meaning that units on top of a hill move much slower than units in a valley.

**NOTE**: In a future version we may change this to ensure there is only a penalty when *moving uphill*, and an increase for moving *downhill*.

### Movement boundary checks

We ensure that units cannot leave a fixed boundary domain $\Omega \in [x_{min}, x_{max}, y_{min}, y_{max}]$:

\begin{equation}
\mathcal{B}(\mathbf{u})=\begin{cases}
x_{min} & u_x < x_{min} \\
x_{max} & u_x > x_{max} \\
y_{min} & u_y < y_{min} \\
y_{max} & u_y > y_{max} 
\end{cases}
\end{equation}

## Attack rolls

### Hit probability

Let $a_i \in [0, 1]$ be the base accuracy modifier of unit $i$, and let $\mathcal{H} \in \mathbb{R}$ be some global hit penalty. Then a chance to perform a successful hit $h_i(t)$ at time $t$ is:

\begin{align}
h_i(t) = \left[ a_i \left(1-a_j^{-1} \right) \left(1-\frac{\mathcal{D}(\mathbf{u}_i, \mathbf{u}_j)}{\mathcal{H}} \right) \right]
\end{align}

where $a_j^{-1} \in [0, 1]$ is the *dodge chance* of enemy unit $j$. here we see that the further a unit is away, the lower the resulting hit chance will be. The fun doesn't stop there though, the attacker now needs to overcome a random roll from a *uniform distribution*:

### Determining damage output

Let $d_j$ be the base damage modifier for unit $j$ and let $v_i(t)$ be the *vitality* or health of unit $i$ at time $t$. Then the initial damage calculation is:

\begin{align}
\dot{d}_j(t)=d_j \left(1+ \frac{z_j(t)-z_i(t)}{2} \right) \quad \text{s.t.} \,\, h_j(t) > \mathcal{U}(0,1)
\end{align}

once again, height plays a significant role in reducing or increasing initial role damage output. However there is another variable inserted into play: *armour*.

### Armour considerations

Let $A_i(t)$ be the amount of armour unit $i$ has remaining at time $t$, then it's difference for step $t$ is calculated as:

$$
A_i(t+1) = A_i(t) - \dot{d}_j(t)
$$

Then using this difference, we determine whether health is affected:

$$
v_i(t+1)=
\begin{cases}
A_i(t+1) > 0 & v_i(t) \\
A_i(t+1) \le 0 \text{ and } A_i(t) > 0 & v_i(t) - A_i(t+1) \\
\text{else} & v_i(t) - \dot{d}_j(t)
\end{cases}
$$

This calculates such that the overflow over from lost armour is also deducted from the vitality pool.

## Terrains

Prior to v0.3.6, terrains were calculated using a mixture of Gaussian densities. This was slow and expensive to compute and a significant bottleneck prior to even running the simulation. From 0.3.6 onwards we generate terrains using **Perlin noise**. This is achieved in the following steps.

#### Smooth noise

Firstly we define a uniformly-distributed matrix $\mathbf{U} \in \mathbb{R}^{X, Y}$ sampled from the uniform distribution $\mathcal{U}(0, 1)$.

Smooth noise for a given position $(x, y)$ within domain $(X, Y)$ is calculated as:

$$N_s(x, y)=f_x f_y U_{x_0, y_0} + (1-f_x) f_y U_{y_0,x_1} + f_x (1-f_y) U_{y_1, x_0} + (1 - f_x) (1-f_y) U_{y_1, x_1}$$

where $f_x= x- \text{int}(x)$ is the fractional part of $x$, and similarly with $f_y$. The indices by which the uniform random matrix $\bf U$ are accessed are labelled $(x_0, x_1, y_0, y_1)$. They are calculated as:
$$x_0=\text{int}(x) + X \, \% \, X \\
y_0=\text{int}(y) + Y \, \% \, Y \\
x_1=(x_0 + X - 1) \, \% \, X \\
y_1=(y_0 + Y - 1) \, \% \, Y \\
$$

#### Turbulence

We can add turbulence to the height noise at $(x, y)$, utilising a provided scale parameter $S_0$ via iteration:

1. Set values $v=0$, $S_n=S_0$.
1. While $S_n>=1$, incrementing $n$ do:
    1. Update $v$ by function $N_s \left(\frac{x}{S_n}, \frac{y}{S_n} \right) \, S_n$
    2. Divide $S_{n+1}=\frac{S_n}{2}$
2. Return $z_{xy}=\frac{128v}{S_0}$

#### Perlin map

We simply iterate $\forall x \in X$ and $\forall y \in Y$, calculating $Z(x, y, S_0)=z_{xy}$. We finally perform minmax scaling to scale matrix $\bf Z$ into the $[0, 1]$ range. This means the lowest map point is 0 and the highest point is 1. By default we use $S_0=\frac{X}{3}$ to give nice-looking scaled maps.

## Tiling

In order to determine the height location of each unit, it holds an index to the (x, y) coordinate of the height map $\bf Z$. Prior to v0.3.6 these indices were calculated using the difference between the units position $x_i$ and a mesh grid of the height domain. This was highly expensive and around 50\% of the performance increase is attributable to this change.

Now we use **lerp** or Linear intERPolation to approximate the best tile index for every unit. Given the height map $\bf Z$ existing within domain bounds $\Omega$ where the range $(A_0, A_1)$ describes real-space and $(B_0, B_1)$ describes index-space, we can linearly map between domains $A \to B$ given value $x_i$ as:

$$
\hat{x}_i = A_1 + (x_i - A_0) \left(\frac{B_1-A_1}{B_0 - A_0} \right)
$$

where in this case the $A$ domain is the x-axis and $B$ domain is the *index* axis, this is repeated for the y-dimension. This means $\hat{x}_i$ must be truncated to an integer so as to access such that a unit's height position is $z_i = Z_{\hat{x}_i, \hat{y}_i}$ like so.

## Unit in range

Let $r_i$ be the base range modifier of unit $i$.

To determine whether an enemy unit is within range of an attack, an *adjusted* range calculation must be made, factoring in the terrain height of the current unit:

\begin{align}
\bar{r}_i = r_i \left(\frac{z_i^2(t)}{3}+1 \right) \tag{8}
\end{align}

where $z_i(t)$ is the units' current $z$ height coordinate. This means for larger values of $z_i^2(t)$, the adjusted range is exponentially higher.

## Deciding how to respond: AI


### Aggressive AI

\begin{align}
\begin{cases}
\mathcal{D}(\mathbf{u}_i, \mathbf{u}_j) > \bar{r}_i & \mathbf{u}_i(t) +\frac{d\mathbf{u}_i}{dt} \\
\text{else} & \text{attack roll}
\end{cases}
\tag{9}
\end{align}

i.e when the distance is greater than the range capability of unit $i$, we need to move closer in order to engage.

### Hit and Run Tactic AI

Currently this AI only activates under the following conditions:


\begin{align}
\begin{cases}
s_i > s_j, \, \bar{r}_i > \bar{r}_j & \text{Hit and Run} \\
\text{else} & \text{Aggressive}
\end{cases}
\end{align}

i.e the speed of unit $i$ is greater than the enemy, and the adjusted range is greater such that this unit cannot simply be ranged down by an opponent. This hit and run is perfect for fast/nimble and longer-range elite units.

\begin{align}
\begin{cases}
\mathcal{D}(\mathbf{u}_i, \mathbf{u}_j) > \bar{r}_i & \mathbf{u}_i(t) + \dot{\mathbf{u}}_i(t) \\
\mathcal{D}(\mathbf{u}_i, \mathbf{u}_j) < \bar{r}_j & \mathbf{u}_i(t) - \dot{\mathbf{u}}_i(t) \\
\text{else} & \text{attack roll}
\end{cases}
\tag{9}
\end{align}

For case one, the unit moves closer to it's opponent when it's out of range, **but** moves away from it's opponent when the oppoenent is within range of itself, leaving the window opportunity when itself is close enough but the opponent is not, maximising potential damage output.